# Project of Advanced Machine Learning

## Import Libraries

In [ ]:
!pip install -q tensorflow-model-optimization
!pip install -U keras-tuner

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import csv
import logging
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import tarfile
import PIL
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential, clone_model, load_model
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.metrics import AUC
from kerastuner.tuners import RandomSearch
from tensorflow_model_optimization.sparsity import keras as sparsity

tensorflow.get_logger().setLevel(logging.ERROR)

## Utility functions

In [ ]:
def get_image_names(tgz):
    with tarfile.open(tgz) as file:
        return [i.name for i in file.getmembers() if i.isfile()]

def show_random_images(n=4, size=10, label=None):
    if(label != None):
        images = df[df['label'] == str(label)].sample(n=(n*n))
    else:
        images = df.sample(n=(n*n))
        
    plt.figure(figsize=(size, size))
    for index, path in enumerate(images['id'].values):
        plt.subplot(n, n, index+1)
        plt.imshow(PIL.Image.open(DATA_PATH + 'images/' + path))
        plt.title('Label: ' + str(images['label'].values[index]))
        plt.axis('off')
    plt.show()

def show_random_images_aug(aug, n=2, size=10):
    x, y = aug.next()
    
    plt.figure(figsize=(size, size))
    for i in range(0, (n*n)):
        plt.subplot(n, n, i+1)
        plt.imshow(x[i])
        plt.title('Label: ' + str(np.where(y[i] == 1)[0][0]))
        plt.axis('off')
    plt.show()
    
def plot_history(history):
    plt.figure(figsize=(10,5))
    plt.plot(history.history['accuracy'], 'orange', label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], 'royalblue', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.figure(figsize=(10,5))
    plt.plot(history.history['precision'], 'orange', label='Training Precision')
    plt.plot(history.history['val_precision'], 'royalblue', label='Validation Precision')
    plt.title('Training and Validation Precision')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend()

    plt.figure(figsize=(10,5))
    plt.plot(history.history['recall'], 'orange', label='Training Recall')
    plt.plot(history.history['val_recall'], 'royalblue', label='Validation Recall')
    plt.title('Training and Validation Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()

    plt.figure(figsize=(10,5))
    plt.plot(history.history['f1_score'], 'orange', label='Training F1-Score')
    plt.plot(history.history['val_f1_score'], 'royalblue', label='Validation F1-Score')
    plt.title('Training and Validation F1-Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1-Score')
    plt.legend()

    plt.figure(figsize=(10,5))
    plt.plot(history.history['loss'], 'orange', label='Training Loss')
    plt.plot(history.history['val_loss'], 'royalblue', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

def true_positives(y_true, y_pred):
    return tensorflow.keras.backend.sum(tensorflow.keras.backend.round(tensorflow.keras.backend.clip(y_true * y_pred, 0, 1)))

def precision(y_true, y_pred):
    predicted_positives = tensorflow.keras.backend.sum(tensorflow.keras.backend.round(tensorflow.keras.backend.clip(y_pred, 0, 1)))

    return true_positives(y_true, y_pred) / (predicted_positives + tensorflow.keras.backend.epsilon())

def recall(y_true, y_pred):
    possible_positives = tensorflow.keras.backend.sum(tensorflow.keras.backend.round(tensorflow.keras.backend.clip(y_true, 0, 1)))

    return true_positives(y_true, y_pred) / (possible_positives + tensorflow.keras.backend.epsilon())

def f1_score(y_true, y_pred):
    possible_positives = tensorflow.keras.backend.sum(tensorflow.keras.backend.round(tensorflow.keras.backend.clip(y_true, 0, 1)))
    predicted_positives = tensorflow.keras.backend.sum(tensorflow.keras.backend.round(tensorflow.keras.backend.clip(y_pred, 0, 1)))

    return 2*(precision(y_true, y_pred)*recall(y_true, y_pred))/(precision(y_true, y_pred)+recall(y_true, y_pred)+tensorflow.keras.backend.epsilon())

## Configuration parameters

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_PATH = '/content/drive/My Drive/data/' # local run only: './data/'

plt.style.use('dark_background')

TEST_RATIO = 0.3
VALIDATION_RATIO = 0.2
NUM_CLASSES = 102
IMG_SIZE = 250
IMG_CHANNELS = 3
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

SEED = 42
np.random.seed(SEED)

VERBOSE = 1
BATCH_SIZE = 32
PATIENCE = 5

CHECKPOINT = ModelCheckpoint(
    (DATA_PATH + 'model.hdf5'), 
    monitor=['val_accuracy'],
    verbose=VERBOSE, 
    mode='max'
)
EARLYSTOP = EarlyStopping(
    monitor='val_accuracy', 
    patience=PATIENCE, 
    restore_best_weights=True
)

## Base models

In [ ]:
model_vgg = VGG16(
    weights='imagenet', 
    include_top=False, 
    input_shape=INPUT_SHAPE
)
model_vgg.trainable = False

model_vgg.summary()

model_xception = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=INPUT_SHAPE
)
model_xception.trainable = False

model_xception.summary()

model_resnet = ResNet50V2(
    weights='imagenet', 
    include_top=False, 
    input_shape=INPUT_SHAPE
)
model_resnet.trainable = False

model_resnet.summary()

## Import data

In [ ]:
df = pd.DataFrame()
df['id'] = sorted(get_image_names(DATA_PATH + 'images.tgz'))
df['label'] = scipy.io.loadmat(DATA_PATH + 'labels.mat')['labels'][0] - 1
df['label'] = df['label'].astype('str')

# tarfile.open(DATA_PATH + 'images.tgz').extractall(DATA_PATH + 'images/') # only first run

In [ ]:
print('Size of dataset:\n', df.count())

print('\nList of images:\n', os.listdir(DATA_PATH + 'images/jpg')[:10])

print('\nDataframe:\n', df.head(10))

print('\nNumber of classes:\n', df['label'].nunique())

print('\nImages for each class:\n', df['label'].value_counts())

print('\nSample images:\n')
show_random_images()

## Dataset split

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(
    df['id'], 
    df['label'], 
    test_size=TEST_RATIO, 
    random_state=SEED, 
    stratify=df['label']
)

train_X, validation_X, train_y, validation_y = train_test_split(
    train_X, 
    train_y, 
    test_size=(VALIDATION_RATIO/(1-TEST_RATIO)), 
    random_state=SEED,
    stratify=train_y
)

train = pd.DataFrame(train_X)
train['label'] = train_y

validation = pd.DataFrame(validation_X)
validation['label'] = validation_y

test = pd.DataFrame(test_X)
test['label'] = test_y

In [ ]:
print('Size of train set:\n', train.shape)

print('\nDataframe of the train set:\n', train.head(10))

print('\nImages for each class of the train set:\n', train['label'].value_counts())

print('\nSize of validation set:\n', validation.shape)

print('\nDataframe of the validation set:\n', validation.head(10))

print('\nImages for each class of the validation set:\n', validation['label'].value_counts())

print('\nSize of test set:\n', test.shape)

print('\nDataframe of the test set:\n', test.head(10))

print('\nImages for each class of the test set:\n', test['label'].value_counts())

## Data preprocessing

In [ ]:
pd.DataFrame(df['label'].value_counts(sort=True)).plot(kind='barh', figsize=(10, 20), color='royalblue')

In [ ]:
pd.DataFrame(train['label'].value_counts(sort=True)).plot(kind='barh', figsize=(10, 20), color='royalblue')

In [ ]:
pd.DataFrame(validation['label'].value_counts(sort=True)).plot(kind='barh', figsize=(10, 20), color='royalblue')

In [ ]:
pd.DataFrame(test['label'].value_counts(sort=True)).plot(kind='barh', figsize=(10, 20), color='royalblue')

### Data augmentation

In [ ]:
train_aug_datagen = ImageDataGenerator(
    rotation_range=45, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    brightness_range=[0.5, 1.5], 
    shear_range=0.15, 
    zoom_range=[0.75, 1.25], 
    fill_mode="nearest", 
    horizontal_flip=True, 
    rescale=1./255
)
train_aug = train_aug_datagen.flow_from_dataframe(
    dataframe=train, 
    directory=(DATA_PATH + 'images/'), 
    x_col='id', 
    y_col='label', 
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE, 
    class_mode='categorical', 
    shuffle=True, 
    seed=SEED
)

validation_aug_datagen = ImageDataGenerator(
    fill_mode="nearest", 
    rescale=1./255
)
validation_aug = validation_aug_datagen.flow_from_dataframe(
    dataframe=validation, 
    directory=(DATA_PATH + 'images/'), 
    x_col='id', 
    y_col='label', 
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE, 
    class_mode='categorical', 
    shuffle=True, 
    seed=SEED
)

test_aug_datagen = ImageDataGenerator(
    fill_mode="nearest", 
    rescale=1./255
)
test_aug = test_aug_datagen.flow_from_dataframe(
    dataframe=test, 
    directory=(DATA_PATH + 'images/'), 
    x_col='id', 
    y_col='label', 
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE, 
    class_mode='categorical', 
    shuffle=True, 
    seed=SEED
)

In [ ]:
print('Sample augmented images of the train set:')
show_random_images_aug(aug=train_aug)

print('Sample augmented images of the validation set:')
show_random_images_aug(aug=validation_aug)

print('Sample augmented images of the test set:')
show_random_images_aug(aug=test_aug)

## Model building

In [ ]:
tensorflow.keras.backend.clear_session()

### Model creation for manual tuning of hyperparameter

In [ ]:
def create_model(BASE_MODEL, LR, DROPOUT_RATE, NUM_UNITS, ACTIVATION, REGULARIZER=None):
    BASE_MODEL_TEMP = clone_model(BASE_MODEL)
    BASE_MODEL_TEMP.set_weights(BASE_MODEL.get_weights())
    BASE_MODEL_TEMP.trainable = True

    # fine_tune_at = int(round(len(BASE_MODEL_TEMP.layers)))
    # for layer in BASE_MODEL_TEMP.layers[:fine_tune_at]:
    #     layer.trainable = False
    
    # for layer in BASE_MODEL_TEMP.layers: 
    #     layer.build(layer.input_shape)

    model = Sequential()
    model.add(BASE_MODEL_TEMP)
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(
        NUM_UNITS, 
        activation=ACTIVATION, 
        kernel_regularizer=REGULARIZER)
    )
    model.add(Dropout(DROPOUT_RATE))
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy', 
        optimizer=Adam(lr=LR), 
        metrics=[precision, recall, f1_score, AUC(), 'accuracy']
    )
    
    return model

### Model creation for automatic tuning of hyperparameter

In [ ]:
def build_model(hp):
    BASE_MODEL_TEMP = clone_model(model_resnet)
    BASE_MODEL_TEMP.set_weights(model_resnet.get_weights())
    BASE_MODEL_TEMP.trainable = True

    model = Sequential()
    model.add(BASE_MODEL_TEMP)
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(
        hp.Int('NUM_UNITS', 768, 1024, step=128, default=768),
        activation=hp.Choice(f'ACTIVATION', ['elu', 'relu']), 
        kernel_regularizer=l2(3e-1))
    )
    model.add(Dropout(hp.Float(f'DROPOUT_RATE', 0.4, 0.7, step=0.1, default=0.5)))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=Adam(
            lr=hp.Choice(f'LR', [1e-5, 2e-5]), 
            beta_1=hp.Float(f'BETA_1', 0.85, 0.95, step=0.05, default=0.9), 
            beta_2=hp.Float(f'BETA_2', 0.9987, 0.9999, step=0.0004, default=0.9991)
        ), 
        metrics=[precision, recall, f1_score, AUC(), 'accuracy']
    )
  
    return model

## Model training

### Naive approach

In [ ]:
PARAMS = {
    'BASE_MODEL': [{'MODEL': model_resnet, 'NAME': 'RESNET50V2'}], # [{'MODEL': model_vgg, 'NAME': 'VGG16'}, {'MODEL':  model_xception, 'NAME': 'XCEPTION'}, {'MODEL': model_resnet, 'NAME': 'RESNET50V2'}],
    'LR': [2e-5], # [1e-3, 1e-4],
    'EPOCHS': [50],
    'DROPOUT_RATE': [0.45], # [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
    'NUM_UNITS': [256], # [128, 256, 512],
    'ACTIVATION': ['elu'], # ['elu', 'relu'],
    'REGULARIZER': [{'REGULARIZER': l2(3e-1), 'NAME': 'L2(0.3)'}] # [{'REGULARIZER': l1(5e-3), 'NAME': 'L1(0.005)'}, {'REGULARIZER': l2(1e-2), 'NAME': 'L2(0.01)'}, {'REGULARIZER': l1(5e-4), 'NAME': 'L1(0.0005)'}, {'REGULARIZER': l2(1e-3), 'NAME': 'L2(0.001)'}]
}

with open(DATA_PATH + 'results.csv', mode='w') as results:
    results_writer = csv.writer(
        results, delimiter=',',
        quotechar='"', 
        quoting=csv.QUOTE_MINIMAL
    )
    results_writer.writerow([
        'EPOCHS', 
        'BATCH SIZE', 
        'BASE MODEL', 
        'DROPOUT RATE', 
        'LR', 
        'NUM UNITS', 
        'ACTIVATION', 
        'REGULARIZER', 
        'LOSS', 
        'ACCURACY', 
        'PRECISION', 
        'RECALL', 
        'F1-SCORE', 
        'VALIDATION LOSS', 
        'VALIDATION ACCURACY', 
        'VALIDATION PRECISION', 
        'VALIDATION RECALL', 
        'VALIDATION F1-SCORE'
    ])

for EPOCHS in PARAMS['EPOCHS']:
    for BASE_MODEL in PARAMS['BASE_MODEL']:
        for DROPOUT_RATE in PARAMS['DROPOUT_RATE']:
            for REGULARIZER in PARAMS['REGULARIZER']:
                for LR in PARAMS['LR']:
                    for NUM_UNITS in PARAMS['NUM_UNITS']:
                        for ACTIVATION in PARAMS['ACTIVATION']:
                            with open(DATA_PATH + 'results.csv', mode='a') as results:
                                results_writer = csv.writer(
                                    results, 
                                    delimiter=',', 
                                    quotechar='"', 
                                    quoting=csv.QUOTE_MINIMAL
                                )

                                model = create_model(
                                    BASE_MODEL['MODEL'], 
                                    LR, 
                                    DROPOUT_RATE, 
                                    NUM_UNITS, 
                                    ACTIVATION, 
                                    REGULARIZER['REGULARIZER']
                                )
                                model.summary()

                                history = model.fit(
                                    train_aug,
                                    validation_data=validation_aug,
                                    epochs=EPOCHS,
                                    callbacks=[CHECKPOINT, EARLYSTOP],
                                    verbose=VERBOSE
                                )

                                print(
                                    '[INFO] Model:\n\tBase model: ', BASE_MODEL['NAME'], 
                                    '\n\tEpochs: ', EPOCHS, 
                                    '\n\tLearning rate: ', LR, 
                                    '\n\tBatch size: ', BATCH_SIZE, 
                                    '\n\tNum units: ', NUM_UNITS, 
                                    '\n\tActivation: ', ACTIVATION, 
                                    '\n\tDropout rate: ', DROPOUT_RATE, 
                                    '\n\tRegularizer: ', REGULARIZER['NAME']
                                )
                                print(
                                    '[WEIGHTS] Model:\n\tSum of the weights of first dense layer: ', sum(sum(abs(model.layers[3].get_weights()[0]))), 
                                    '\n\tSum of the biases of first dense layer: ', sum(abs(model.layers[3].get_weights()[1])), 
                                    '\n\tSum of the weights of second dense layer: ', sum(sum(abs(model.layers[5].get_weights()[0]))), 
                                    '\n\tSum of the biases of second dense layer: ', sum(abs(model.layers[5].get_weights()[1]))
                                )
                                print(
                                    '[TRAINING] Model:\n\tTrain loss: ', history.history['loss'][-1], 
                                    '\n\tTrain accuracy: ', history.history['accuracy'][-1], 
                                    '\n\tTrain precision: ', history.history['precision'][-1], 
                                    '\n\tTrain recall: ', history.history['recall'][-1], 
                                    '\n\tTrain f1-score: ', history.history['f1_score'][-1]
                                )
                                print(
                                    '[VALIDATING] Model:\n\tValidation loss: ', history.history['val_loss'][-1], 
                                    '\n\tValidation accuracy: ', history.history['val_accuracy'][-1], 
                                    '\n\tValidation precision: ', history.history['val_precision'][-1], 
                                    '\n\tValidation recall: ', history.history['val_recall'][-1], 
                                    '\n\tValidation f1-score: ', history.history['val_f1_score'][-1]
                                )
                                plot_history(history)

                                results_writer.writerow([
                                    EPOCHS, 
                                    BATCH_SIZE, 
                                    BASE_MODEL['NAME'], 
                                    DROPOUT_RATE, 
                                    LR, 
                                    NUM_UNITS, 
                                    ACTIVATION, 
                                    REGULARIZER['NAME'], 
                                    history.history['loss'][-1], 
                                    history.history['accuracy'][-1], 
                                    history.history['precision'][-1], 
                                    history.history['recall'][-1], 
                                    history.history['f1_score'][-1], 
                                    history.history['val_loss'][-1], 
                                    history.history['val_accuracy'][-1], 
                                    history.history['val_precision'][-1],
                                    history.history['val_recall'][-1], 
                                    history.history['val_f1_score'][-1]]
                                )
                                model.save(DATA_PATH + 'model_' + str(EPOCHS) + '_' + str(BATCH_SIZE) + '_' + BASE_MODEL['NAME'] + '_' + str(DROPOUT_RATE) + '_' + str(LR) + '_' + str(NUM_UNITS) + '_' + ACTIVATION + '_' + REGULARIZER['NAME'] + '.hdf5')                        
                                
                                tensorflow.keras.backend.clear_session()

### Structured approach

In [ ]:
EPOCHS = 10
MAX_TRIALS = 64

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    executions_per_trial=1,
    directory=DATA_PATH,
    project_name='tuner'
)

tuner.search(
    train_aug,
    validation_data=validation_aug,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[
        EarlyStopping(
        monitor='val_accuracy',
        patience=PATIENCE,
        restore_best_weights=True)
    ],
    verbose=VERBOSE
)

tuner.results_summary(num_trials=MAX_TRIALS)

In [ ]:
best_model = tuner.get_best_models()[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
best_model.save(DATA_PATH + 'best_model.hdf5')
best_model = load_model(
    DATA_PATH + 'best_model.hdf5',
    custom_objects={
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }
)
best_model.summary()

### Pruning

In [ ]:
EPOCHS = 30
BASE_MODEL = clone_model(model_resnet)
BASE_MODEL.set_weights(model_resnet.get_weights())
INITIAL_SPARSITY = 0.2
FINAL_SPARSITY = 0.8
NUM_UNITS = 896
DROPOUT_RATE = 0.6
ACTIVATION = 'elu'
L2_FACTOR = 3e-1
LR = 2e-5
BETA_1 = 0.95
BETA_2 = 0.9987

pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(
        initial_sparsity=INITIAL_SPARSITY,
        final_sparsity=FINAL_SPARSITY,
        begin_step=0,
        end_step=(np.ceil(1.0*train.shape[0]/BATCH_SIZE).astype(np.int32)*EPOCHS),
        frequency=100
    )
}

pruned_model = Sequential([
    BASE_MODEL,
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    sparsity.prune_low_magnitude(Dense(
        NUM_UNITS, 
        activation=ACTIVATION, 
        kernel_regularizer=l2(L2_FACTOR)
    ), **pruning_params),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

pruned_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(
        lr=LR, 
        beta_1=BETA_1, 
        beta_2=BETA_2
    ),
    metrics=[precision, recall, f1_score, AUC(), 'accuracy']
)

history = pruned_model.fit(
    train_aug,
    validation_data=validation_aug,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=VERBOSE,
    callbacks=[
        sparsity.UpdatePruningStep(),
        sparsity.PruningSummaries(log_dir=(DATA_PATH+'pruning/')),
    ]
)

plot_history(history)

In [ ]:
pruned_model = sparsity.strip_pruning(pruned_model)

pruned_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(
        lr=LR, 
        beta_1=BETA_1, 
        beta_2=BETA_2
    ),
    metrics=[precision, recall, f1_score, AUC(), 'accuracy']
)

pruned_model.save(DATA_PATH + 'pruned_model.hdf5')
pruned_model = load_model(
    DATA_PATH + 'pruned_model.hdf5',
    custom_objects={
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }
)
pruned_model.summary()

for i, w in enumerate(pruned_model.get_weights()):
    if i == 270:
        print("{} - Total params: {}, Zeros: {:.2f}%".format(pruned_model.weights[i].name, w.size, np.sum(w==0)/w.size*100))

## Model testing

In [ ]:
score_best_model = best_model.evaluate(test_aug, verbose=VERBOSE)
score_pruned_model = pruned_model.evaluate(test_aug, verbose=VERBOSE)

print(
    '\n[TESTING] Best Model:\n\tTest loss: ', score_best_model[0], 
    '\n\tTest accuracy: ', score_best_model[5], 
    '\n\tTest precision: ', score_best_model[1], 
    '\n\tTest recall: ', score_best_model[2], 
    '\n\tTest f1-score: ', score_best_model[3], 
    '\n\tTest auc: ', score_best_model[4],
    '\n\n[TESTING] Pruned Model:\n\tTest loss: ', score_pruned_model[0], 
    '\n\tTest accuracy: ', score_pruned_model[5], 
    '\n\tTest precision: ', score_pruned_model[1], 
    '\n\tTest recall: ', score_pruned_model[2], 
    '\n\tTest f1-score: ', score_pruned_model[3], 
    '\n\tTest auc: ', score_pruned_model[4]
)